# FORGE Seismic Events 3D Visualization

This notebook creates interactive 3D scatter plots of seismic events from the Utah FORGE geothermal field. It processes multiple CSV catalog files and generates both individual plots for each catalog and a combined visualization of all events.

## Output Options
- Interactive plots for exploration (set `save_images = False`)
- High-resolution PNG files (set `save_images = True`)

## Summary and Usage Notes

### 📊 **Generated Visualizations:**
1. **Individual Catalog Plots**: One 3D plot per CSV file showing temporal subsets
2. **Combined Plot**: All events from all catalogs in a single visualization

### 🎨 **Visualization Features:**
- **3D Scatter Plot**: X-Y-Depth positioning of seismic events
- **Magnitude Color Coding**: Warmer colors = higher magnitudes
- **Consistent Scaling**: All plots use the same axis ranges and color scales
- **Interactive Exploration**: Zoom, rotate, and hover for detailed information

### ⚙️ **Configuration Options:**
- **Display Mode**: Set `save_images = False` for interactive plots, `True` for file output
- **Output Quality**: Configurable DPI and dimensions for publication use
- **File Filtering**: Automatically processes all `FORGE*.csv` files in the input directory

### 📁 **Output Files:**
- Individual plots: `{catalog_name}.png` 
- Combined plot: `all_events_combined.png`
- All saved to the `figures/3D_plots/` directory

### 🔧 **Data Processing:**
- Automatic column name cleaning (removes spaces, strips whitespace)
- Invalid depth filtering (removes events with depth = 0)
- Global range calculation ensures visual consistency across plots

This notebook provides a comprehensive 3D visualization workflow for analyzing seismic event spatial distributions and magnitude patterns in the FORGE geothermal field.

In [ ]:
# Import required libraries for data processing and 3D visualization
import pandas as pd
import glob
import plotly.express as px
import os
import numpy as np

def generate_tick_values(min_val, max_val, step=0.5):
    """
    Generate evenly spaced tick values for color bar with specified step size.
    
    Args:
        min_val (float): Minimum value in the range
        max_val (float): Maximum value in the range  
        step (float): Step size between ticks (default: 0.5)
    
    Returns:
        list: Sorted list of tick values including min/max boundaries
    """
    values = np.arange(
        np.ceil(min_val / step) * step,
        np.floor(max_val / step) * step + step, step).tolist()
    values = sorted(set(values + [min_val, max_val]))
    return values

print("✅ Libraries imported and utility functions defined")

In [ ]:
# Configuration Parameters
folder_path = 'GES16Aand16BStimulationMonitoringApril2024/**'  # Path to CSV catalog files
output_path = 'figures/3D_plots/'                          # Directory for saved plots
save_images = False                                            # True: save PNG files, False: show interactive plots
width_mm = 85                                                  # Figure width in millimeters (journal standard)
dpi = 300                                                      # Resolution for saved images

print(f"📁 Input directory: {folder_path}")
print(f"💾 Output directory: {output_path}")
print(f"🖼️  Mode: {'Save images' if save_images else 'Interactive display'}")
print(f"📏 Figure size: {width_mm}mm @ {dpi} DPI")

In [ ]:
# Data Discovery and Global Range Calculation
print("🔍 Discovering CSV catalog files...")

# Find all FORGE catalog files in the specified directory
csv_files = glob.glob(os.path.join(folder_path, "FORGE*.csv"))
os.makedirs(output_path, exist_ok=True)

print(f"📊 Found {len(csv_files)} catalog files")

# Initialize variables to track global min/max values across all files
# This ensures consistent axis ranges and color scales for all plots
global_min_mag = float('inf')
global_max_mag = float('-inf')
global_min_x = float('inf')
global_max_x = float('-inf')
global_min_y = float('inf')
global_max_y = float('-inf')
global_min_depth = float('inf')
global_max_depth = float('-inf')

print("📏 Calculating global data ranges...")

# First pass: scan all files to determine overall data ranges
for i, datapath in enumerate(csv_files):
    data = pd.read_csv(datapath)
    data.columns = data.columns.str.strip().str.replace(' ', '_')  # Clean column names
    data = data[data['Depth'] != 0]  # Filter out invalid depth values
    
    # Update global ranges
    global_min_mag = min(global_min_mag, data['MomMag'].min())
    global_max_mag = max(global_max_mag, data['MomMag'].max())
    global_min_x = min(global_min_x, data['X'].min())
    global_max_x = max(global_max_x, data['X'].max())
    global_min_y = min(global_min_y, data['Y'].min())
    global_max_y = max(global_max_y, data['Y'].max())
    global_min_depth = min(global_min_depth, data['Depth'].min())
    global_max_depth = max(global_max_depth, data['Depth'].max())
    
    if (i + 1) % 5 == 0:  # Progress indicator
        print(f"  Processed {i + 1}/{len(csv_files)} files...")

print(f"\n📊 Global Data Ranges:")
print(f"  • Magnitude: {global_min_mag:.3f} to {global_max_mag:.3f}")
print(f"  • X coordinates: {global_min_x:.1f} to {global_max_x:.1f} m")
print(f"  • Y coordinates: {global_min_y:.1f} to {global_max_y:.1f} m") 
print(f"  • Depth range: {global_min_depth:.1f} to {global_max_depth:.1f} m")

## Individual Catalog Visualizations

Generate 3D scatter plots for each individual catalog file. All plots use consistent:
- **Axis ranges**: Same X, Y, and depth limits across all plots
- **Color scaling**: Unified magnitude color bar for comparison
- **Camera angle**: Standard viewing perspective for consistency
- **Marker styling**: Uniform size and transparency

In [ ]:
# Generate Individual 3D Plots for Each Catalog
print("🎨 Creating individual 3D plots...")

plot_count = 0
for i, datapath in enumerate(csv_files):
    data = pd.read_csv(datapath)
    
    if not data.empty:
        # Data preprocessing
        data.columns = data.columns.str.strip().str.replace(' ', '_')
        data = data[data['Depth'] != 0]  # Remove invalid depth entries
        title = os.path.basename(datapath)
        
        # Create 3D scatter plot with consistent styling
        fig = px.scatter_3d(
            data,
            x='X', y='Y', z='Depth',
            color='MomMag',                              # Color by magnitude
            range_color=[global_min_mag, global_max_mag], # Consistent color scale
            size=[1] * len(data),                        # Uniform marker size
            title=title,
            labels={
                'X': 'X Coordinate (m)',
                'Y': 'Y Coordinate (m)', 
                'Depth': 'Depth (m)',
                'MomMag': 'Moment Magnitude'
            },
            size_max=14,
            opacity=0.7
        )
        
        # Customize marker appearance and layout
        fig.update_traces(marker=dict(line=dict(width=0)))  # Remove marker outlines
        tickvals = generate_tick_values(global_min_mag, global_max_mag)
        
        fig.update_layout(
            width=800, height=600,
            title_x=0.5,
            margin=dict(l=40, r=40, b=40, t=40),
            scene=dict(
                zaxis=dict(range=[global_min_depth, global_max_depth][::-1]),  # Invert depth axis
                xaxis=dict(range=[global_min_x, global_max_x]),
                yaxis=dict(range=[global_min_y, global_max_y]),
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),  # Standard viewing angle
                aspectmode="cube"  # Equal aspect ratios
            ),
            coloraxis_colorbar=dict(
                title="Moment Magnitude",
                title_side="top", 
                tickvals=tickvals,
                tickmode='array',
                ticks="outside",
                ticklen=6,
                tickwidth=1
            )
        )
        
        # Display or save the plot
        if not save_images:
            fig.show()
        else:
            width_in = width_mm / 25.4
            height_in = width_in * 0.75
            image_path = os.path.join(output_path, f"{os.path.basename(datapath).strip('.csv')}.png")
            fig.write_image(image_path, width=int(width_in * dpi), height=int(height_in * dpi), scale=2)
            
        plot_count += 1
        if plot_count % 5 == 0:
            print(f"  Generated {plot_count}/{len(csv_files)} plots...")

print(f"✅ Generated {plot_count} individual catalog plots")

## Combined Events Visualization

Create a single 3D plot containing all seismic events from all catalog files.

In [ ]:
# Generate Combined Plot of All Events
print("🌍 Creating combined visualization of all events...")

# Concatenate all CSV files into a single dataset
data = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)
data.columns = data.columns.str.strip().str.replace(' ', '_')
data = data[data['Depth'] != 0]  # Remove invalid depth entries

# Create informative title with date range
try:
    start_date = data['Trig_Date'].iloc[0].strip()
    end_date = data['Trig_Date'].iloc[-1].strip()
    title = f"All Seismic Events: {start_date} to {end_date}"
except:
    title = "All Seismic Events - Combined Catalog"

print(f"📊 Combined dataset: {len(data)} events total")

# Create comprehensive 3D scatter plot
fig = px.scatter_3d(
    data,
    x='X', y='Y', z='Depth',
    color='MomMag',
    range_color=[global_min_mag, global_max_mag],
    size=[1] * len(data),
    title=title,
    labels={
        'X': 'X Coordinate (m)',
        'Y': 'Y Coordinate (m)',
        'Depth': 'Depth (m)', 
        'MomMag': 'Moment Magnitude'
    },
    size_max=14,
    opacity=0.7
)

# Apply consistent styling matching individual plots
fig.update_traces(marker=dict(line=dict(width=0)))
tickvals = generate_tick_values(global_min_mag, global_max_mag)

fig.update_layout(
    width=800, height=600,
    title_x=0.5,
    margin=dict(l=40, r=40, b=40, t=40),
    scene=dict(
        zaxis=dict(range=[global_min_depth, global_max_depth][::-1]),
        xaxis=dict(range=[global_min_x, global_max_x]),
        yaxis=dict(range=[global_min_y, global_max_y]),
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
        aspectmode="cube"
    ),
    coloraxis_colorbar=dict(
        title="Moment Magnitude",
        title_side="top",
        tickvals=tickvals,
        tickmode='array',
        ticks="outside",
        ticklen=6,
        tickwidth=1
    )
)

# Display or save the combined plot
if not save_images:
    fig.show()
else:
    width_in = width_mm / 25.4
    height_in = width_in * 0.75
    image_path = os.path.join(output_path, "all_events_combined.png")
    fig.write_image(image_path, width=int(width_in * dpi), height=int(height_in * dpi), scale=2)
    print(f"💾 Saved combined plot: {image_path}")

print("✅ Combined visualization complete")